<a href="https://colab.research.google.com/github/ms624atyale/Phonetics/blob/main/18JUL25_ASR_tutorial_wer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 음성인식 오류율 측정 예제

## 환경셋업

In [ ]:
# Run this cell to automatically clone the repo (downloading all source code)
import sys
import os

if 'google.colab' in sys.modules:
    if not os.path.exists("/content/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" /content/a1003
    %cd /content/a1003/local
else:
    if not os.path.exists("~/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" ~/a1003
    %cd ~/a1003/local

Cloning into '/content/a1003'...
remote: Enumerating objects: 356, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 356 (delta 105), reused 104 (delta 35), pack-reused 148 (from 1)
Receiving objects: 100% (356/356), 70.75 MiB | 25.79 MiB/s, done.
Resolving deltas: 100% (160/160), done.
/content/a1003/local


## 개별 문장에 대한 오류율 측정




다음 코드는 두 개의 한국어 문장(정답, 음성인식결과)에 대하여
edit distance(최소편집거리)를 계산하여 WER과 CER을 측정하는 코드입니다.

Edit distance를 계산하기 위해서 `editdistance` 모듈을 import하고,
두 개의 문장을 각 변수에 저장합니다.


In [ ]:
import editdistance

ref_text = "오늘 서울의 날씨가 어때"
hyp_text = "음 오늘의 날씨 가 어때"

WER을 측정하기 위해서 문장을 단어단위로 분리하여 리스트에 저장하빈다.

In [ ]:
ref = ref_text.split()
hyp = hyp_text.split()

print(ref, hyp, sep='\n')

['오늘', '서울의', '날씨가', '어때']
['음', '오늘의', '날씨', '가', '어때']


Edit distance를 계산합니다. `editdistance.eval()` 함수는 두개의 리스트를 입력받아서,
두 리스트 간의 editdistance를 반환하는 함수입니다.

In [ ]:
E = editdistance.eval(ref, hyp) #count all subsitution, deletion, insertion
N = len(ref)
WER = E/N*100
print(f"N={N}, E={E}, WER={WER}")

N=4, E=4, WER=100.0


CER을 측정하기 위해서 문장을 각 글자의 리스트로 변환합니다.

In [ ]:
ref = list(ref_text)
hyp = list(hyp_text)
print(ref, hyp, sep='\n')


글자 단위로 분리된 리스트(`ref`, `hyp`)에 대해서 같은 방법으로 editdistance를 측정합니다.
공백문자도 하나의 글자로 처리된 것도 확인하세요.


In [ ]:
E = editdistance.eval(ref, hyp)
N = len(ref)
CER = E/N*100
print(f"N={N}, E={E}, CER={CER}")

N=4, E=4, CER=100.0


이번에는 공백문자를 무시하고(즉, 띄어쓰기 오류를 완전히 무시하고) CER을 측정해 봅니다.
앞에서 글자단위로 분리한 것과 다르게 이번에는 공백문자를 미리 제거합니다. It is hard to say one is better than the other. As long as you are consistent, it's fine.
>
평가 데이터를 어떻게 볼 것인가? 한국전자통신연구원에서는 숫자, 띄워쓰기 등을 어떻게 반영할 것인가 보고 있다. 요즘은 띄워쓰기를 평가 데이터에 반영하는 분위기고, 숫자의 표현은 글자 혹은 숫자 어떻게 해야 가독성 높아지나 등도 고려


In [ ]:
ref = list(''.join(ref_text.split()))
hyp = list(''.join(hyp_text.split()))
print(ref, hyp, sep='\n')

['오', '늘', '서', '울', '의', '날', '씨', '가', '어', '때']
['음', '오', '늘', '의', '날', '씨', '가', '어', '때']


In [ ]:
E = editdistance.eval(ref, hyp) #Korean has user-dependent space uses, it'd better estimate error rate by character, not word. WER for English at all time.
N = len(ref)
CER = E/N*100
print(f"N={N}, E={E}, CER={CER}")

N=10, E=3, CER=30.0


## 여러 문장에 대한 평균 오류율 측정

지금까지는 한개의 {정답,인식결과} 쌍에 대해서 WER, CER을 측정해보았습니다.
실제로는 매우 많은 {정답,인식결과} 쌍에 대해서 오류율을 계산하고 전체의 오류율을 성능지표로 사용합니다.
이를 위해서 `kaldi`와 `ESPnet`은 다음과 같은 형식으로 여러 파일에 대한 정답전사문과 인식결과를 파일로 저장해둡니다.

In [ ]:
!cat data/ref.txt

KsponSpeech_E00001 어 일단은 억지로 과장해서 이렇게 하는 것보다 진실된 마음으로 이걸 어떻게 전달할 수 있을까 공감을 시킬 수 있을까 해서 좀
KsponSpeech_E00002 혼인 신고를 또 해야 되잖아 
KsponSpeech_E00003 약간 젊은 엄마 같은 느낌이야 
KsponSpeech_E00004 응 근데 오늘 일단 밥 먹고 이것 저것 하다가 시간되면 뭐 가는 거고 안 되면
KsponSpeech_E00005 아 우린 또 그런 거 안 하잖아 어 그치 
KsponSpeech_E00006 아 근데 진짜 감튀 먹고 있는데 데려온 거 심했다
KsponSpeech_E00007 그래서 저승에서 그 애기를 찾아서 막 돌아다니는 내용인데 
KsponSpeech_E00008 그런가 
KsponSpeech_E00009 그 너가 매운 거를 잘 못먹어서 못 먹기도 하고
KsponSpeech_E00010 그래서 나는 지금 7월달에는 사람들이 돈 30만 원씩 뭐 달마다 30만 원씩 모아서 지금 그 태국여행 가기로 했거든 


In [ ]:
!cat data/hyp.txt

KsponSpeech_E00001 어 일단은 억지로 과장에서 이렇게 하는 것보다 진실된 마음으로 이걸 어떻게 전달을 할 수 있을까 공감을 시킬 수 있을까 해서 좀
KsponSpeech_E00002 혼인 신고를 또 해야 되잖아
KsponSpeech_E00003 약간 젊은 엄마 같은 느낌이야
KsponSpeech_E00004 음 근데 오늘 일단 밥 먹고 이것저것 다 따라 하다가 시간 되면 >뭐 가는 거고 안 되면
KsponSpeech_E00005 아무랜도 그런 거 알잖아 어 그지
KsponSpeech_E00006 아 근데 진짜 감기 먹고 있는데 데려온 구심했다
KsponSpeech_E00007 그래서 저순에서 그 애기를 찾아서 막 돌아다니는 내용인데
KsponSpeech_E00008 그런가
KsponSpeech_E00009 근데 너가 매운 거를 잘 못 먹었 못 먹기도 하고
KsponSpeech_E00010 그래도 나는 지금 칠 월 달에는 사람들이 조 삼십만 원씩 뭐 달만 삼십 하십 모아서 지금 그 태홍된가 그랬거든


이 두개의 파일을 읽어서 utternace-id 가 같은 문장간의 editdistance를 출력하는 파이선 코드가 `uttwer.py`, `uttcer.py` 파일입니다.
다음과 같은 방법으로 실행합니다.

In [ ]:
!python uttwer.py data/ref.txt data/hyp.txt

KsponSpeech_E00001 15.00 3 20
KsponSpeech_E00002  0.00 0 5
KsponSpeech_E00003  0.00 0 5
KsponSpeech_E00004 46.67 7 15
KsponSpeech_E00005 66.67 6 9
KsponSpeech_E00006 33.33 3 9
KsponSpeech_E00007 12.50 1 8
KsponSpeech_E00008  0.00 0 1
KsponSpeech_E00009 33.33 3 9
KsponSpeech_E00010 66.67 12 18
N= 99 E= 35 WER= 35.35


In [ ]:
!python uttcer.py data/ref.txt data/hyp.txt

KsponSpeech_E00001  4.00 2 50
KsponSpeech_E00002  0.00 0 11
KsponSpeech_E00003  0.00 0 12
KsponSpeech_E00004 17.24 5 29
KsponSpeech_E00005 42.86 6 14
KsponSpeech_E00006 10.53 2 19
KsponSpeech_E00007  4.17 1 24
KsponSpeech_E00008  0.00 0 3
KsponSpeech_E00009 22.22 4 18
KsponSpeech_E00010 38.30 18 47
N= 227 E= 38 CER= 16.74


다른 데이터에 대해서도 해봅시다.

In [ ]:
!git clone https://hf.co/datasets/pkyoung/a1003.git ./data/a1003
!cp data/a1003/result_espnet_ma16k2401a.txt ./result.txt
!head result.txt



Cloning into './data/a1003'...
remote: Enumerating objects: 206, done.
remote: Total 206 (delta 0), reused 0 (delta 0), pack-reused 206 (from 1)
Receiving objects: 100% (206/206), 46.70 KiB | 7.78 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Filtering content: 100% (180/180), 49.18 MiB | 26.26 MiB/s, done.
spk1/1 위키피디아에 따르면 소리 또는 음 또는 음파는 공기나 물 같은 매질의 진동을 통해 전달되는 종바라고 합니다.
spk1/10 파동은 반복적으로 진동하는 신호를 나타내기 때문에 우는 소리를 웨이폼 형태로 나타낼 수 있습니다.
spk1/11 풀이의 변환을 통해 얻은 스펙트럼은 시간 정보가 없어지게 됩니다.
spk1/12 파형을 일정한 특정 길이의 프레임으로 잘라서 각 프레임마다 프리의 변환을 취하고 스펙트럼을 구하는 방법을 취합니다.
spk1/13 이렇게 음성 전체로부터 얻은 여러 개의 스펙트럼을 시간 축에 나열하면 시간 변화에 따른 스펙트럼의 변화를 스펙트로 그램으로 정의합니다.
spk1/14 스펙트로그램은 소리나 파동을 시각하여 파악하기 위한 도구로 파형과 스펙트럼의 특징이 조합되어 있습니다.
spk1/15 사람의 청각 기관은 고주파보다 저주파 대역에서 더 민감하다고 합니다.
spk1/16 이러한 이유를 달팽이관의 구조로 살펴보면 달팽이관의 가장 안쪽 청각 세포는 저주파 대역을 인지합니다.
spk1/17 바깥쪽 청각세포는 고주파 대역을 인지한다는 사실을 통해 모든 주파수 대역을 같은 비중으로 인지하지 않습니다.
spk1/18 고주파에서 저주파로 내려갈수록 담당하는 주파수 대역이 점점 더 조밀해진다는 점입니다.


In [ ]:
!python uttcer.py data/a1003/text result.txt

spk1/1  4.55 2 44
spk1/10  6.98 3 43
spk1/11 14.81 4 27
spk1/12  6.12 3 49
spk1/13  1.75 1 57
spk1/14  4.35 2 46
spk1/15  3.45 1 29
spk1/16  2.33 1 43
spk1/17  2.17 1 46
spk1/18  2.70 1 37
spk1/19  3.45 1 29
spk1/2  2.78 1 36
spk1/20  2.33 1 43
spk1/3  3.33 1 30
spk1/4  5.00 1 20
spk1/5  2.17 1 46
spk1/6  2.78 1 36
spk1/7  1.85 1 54
spk1/8  5.45 3 55
spk1/9  5.13 2 39
spk2/utt1 18.75 3 16
spk2/utt10 23.08 3 13
spk2/utt11 14.29 2 14
spk2/utt12 29.41 5 17
spk2/utt13 11.43 4 35
spk2/utt14 10.00 1 10
spk2/utt15  5.56 1 18
spk2/utt16 13.33 2 15
spk2/utt17 12.50 3 24
spk2/utt18 27.78 5 18
spk2/utt19 17.86 5 28
spk2/utt2 11.76 2 17
spk2/utt20 10.00 1 10
spk2/utt3  8.33 1 12
spk2/utt4 23.81 5 21
spk2/utt5  6.45 2 31
spk2/utt6  6.67 1 15
spk2/utt7 30.00 6 20
spk2/utt8  8.70 2 23
spk2/utt9 17.95 7 39
spk3/data1  2.86 1 35
spk3/data10 45.00 18 40
spk3/data11  9.52 4 42
spk3/data12  2.78 1 36
spk3/data13  9.68 3 31
spk3/data14 12.90 4 31
spk3/data15 68.97 20 29
spk3/data16  0.00 0 7
spk3/data17  9

문장부호를 제외한 인식률을 측정해봅니다.

In [ ]:
!python uttwer.py <(python pp.py -s data/a1003/text) <(python pp.py -s result.txt) > /dev/null

N= 6835 E= 504 WER=  7.37


화자별 성능도 측정해봅시다.

In [ ]:
%%bash
for s in spk{1..9}; do
  echo -n "$s " >&2
  python uttcer.py data/a1003/text <(grep "$s/" result.txt) > /dev/null
done

spk1 N= 809 E= 32 CER=  3.96
spk2 N= 396 E= 61 CER= 15.40
spk3 N= 610 E= 70 CER= 11.48
spk4 N= 1360 E= 93 CER=  6.84
spk5 N= 981 E= 54 CER=  5.50
spk6 N= 515 E= 39 CER=  7.57
spk7 N= 515 E= 253 CER= 49.13
spk8 N= 848 E= 40 CER=  4.72
spk9 N= 995 E= 67 CER=  6.73
